### Object Detection

1. object localization
    * image classification -> classification with localization -> detection
    * 在图像分类时，神经网络通过softmax可以输出一个图片属于哪一类，这时图片上没有其他物体，比如
        1. 输出类别为行人、汽车、摩托车、背景
        2. 使用softmax来判断属于哪一类
        3. 每一张图片只有一个物体， 即只属于一类
    * 在物体定位时
        1. 令NN多输出四个参数，分表表示框出物体中心和长宽，如图，(bx, by, bh, bw) = (0.5, 0.7, 0.3, 0.4)  
        <img src="定位.png" width="200">
    * 如何定义目标标签y
        1. 需要输出(bx, by, bh, bw)以及类型(1-4)
        2. y = $[P_c, bx, by, bh, bw, c_1, c_2, c_3]$
            1. 其中$p_c$表示图片是否有物体，如果是背景那么为0， 否则为1
            2. bx, by, bh, bw,表示物体的位置参数
            3. c_i 表示实际上属于哪一类物体
        3. loss function
            1. square loss: $L(\hat{y}, y) = \sum_{i=1}^{m}(\hat{y}_i - y_i)^2$ if $p_c = 1$ else $(\hat{y}_0 - y_0)^2$

2. Landmark Detection
    1. 人脸/人形态特征点检测
        * 通过CNN输出各个特征点的位置
        * 输出为$[p_c, lx_1, ly_1, ..., lx_n, ly_n], p_c$表示是否有面部/人体，(lx, ly)表示各个特征点的位置。
        * 每个特征点的表示的相对人体的位置应该相同

3. Object Detection
    1. 物体检测用来圈出一张图片的所有目标对象
    2. Sliding windows detection
        * 通过将图片用滑动窗口分割，判断每个窗口内是否存在一个目标对象
        * 可以根据滑动的strides来控制遍历速度
        * 改变窗口大小，以固定stride重新遍历
        * 通过以上方法，尽量获得所有物体

4. Turning FC layer into conv layer 
    1. 经过池化后的layer，可以使用卷积来代替全连接，即使用多个与上一层相同大小的卷积核，将上一层转化为一维tensor
    2. 滑动窗口的卷积实现
        * 假设训练集图片像素为$14*14*3$, (训练集是单分类，每张图片只有一个物体) 有  
        <img src="slidewindow.png" width="700">
        * 假设测试集像素为$16*16*3$, （测试集是多物体图片）那么  
        <img src="slidewindowstest.png" width="700">  
        图中windows大小为$14*14$（和训练集相同），步长为1， 因此最后得到四个windows，size=$2*2$
        * 假设测试集2大小为$28*28*3$，滑动窗口大小为$14*14$，步长为2，最终得到$8*8$的输出

5. Bounding box predictions
    1. 滑动窗口的缺陷
        * 有可能任意一个窗口都不能完全贴合整个物体
    2. YOLO算法
        * 首先将图片用网格分割
        * 使用图像分类和物体检测对处理每个网格
        * 假设以$3*3$网格分割，那么每个部分输出一个长度为8的向量$(p_c, b_i, c_i)$，总共输出$3*3*8$的向量
        * 物体的位置应该为物体在每个格子中的位置
    3. label向量的编码
        * 标记位和滑动窗口相同
        * 将左上角标记位(0, 0), 右下角为(1, 1), 则$b_x, b_y, b_w, b_h$为目标物体相对于网格的位置, 其中中心点坐标$(b_x, b_y)$必须介于(0, 1), 而长宽标记可以大于1

6. Evaluation
     1. 交并比(Intersection on union, IOU)
         * 交并比表示预测框架与实际框架相交部分与预测框架的面积的比值
         * 当IOU > 0.5时表示框架正确

7. Non-max suppression 非极大值抑制
    1. 当多个网格分割了同一个物体时($b_w, b_h$两个参数大于1)，需要选出物体中心点真实所在的网格
        * 首先选择$p_c$最大的网格，视为真正的物体中心所在网格，获得其框架参数b，将框架高亮
        * 非极大值抑制会抑制那些与高亮框架交并比最大的框架
    2. 使用方法
        * 去掉$p_c \leq 0.6$的所有框架
        * 对于剩下的框架
            1. 选出$p_c$最大的框架作为预测值
            2. 去掉与预测值$IoU \geq 0.5$的所有框架

8. Anchor Boxes
    1. 当试图用一个网格识别出多个物体时，可以使用anchor boxes
    2. anchor boxes通过事先定义两个不同shape的box，然后分别识别不同的物体
    3. 使用anchor boxes后，输出y会扩充至n个anchor boxes的shape
        * 例如，假设有3* 3的网格，输出长度为8，即输出为3* 3* 8，那么在使用两个anchor box的时候，输出变为3* 3* 16